In [36]:
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd
import pyodbc as db
import os

url = "http://www.rhinohockeysiouxfalls.com/"

request = req.get(url).text
soup = bs(request, "lxml")

# Database Connection Settings
serverName = "DESKTOP-8FK7M67\SQLEXPRESS"
databaseName = "RhinoHockey"
connectionString = "Driver={{SQL Server}};Server={};Database={}".format(serverName, databaseName)

# DB Operations
# cnxn = db.connect(connectionString)
# cursor = cnxn.cursor()
# cursor.execute('''DROP TABLE IF EXISTS dbo.Test CREATE TABLE dbo.Test (Team VARCHAR(255), Wins INT, Loses INT)''')
# cursor.commit()
# cursor.close()

In [17]:
import re

#Break into Cenior Rhino tab
base_search = "cenior rhino"
rhino_tab = soup.find('a', string=re.compile(base_search + "$", re.I)).find_next_sibling()

#Search for stats tab
stats_search = "stats"
stats_tab = rhino_tab.find('a', string=re.compile(stats_search + '$', re.I)).find_next_sibling()


full_data = []
# Work down the stats tree to parse all seasons
for season_type in stats_tab.find_all('li', recursive=False):
    season = {}
    season['SeasonType'] = season_type.a.get_text(strip=True)
    season['SeasonTypeUrl'] = season_type.a['href']
    season_list = []
    for overall_season_stats in season_type.find('ul').find_all('li',recursive=False):
        specific_season = {}
        specific_season['Season'] = overall_season_stats.a.get_text(strip=True)
        specific_season['SeasonUrl'] = overall_season_stats.a['href']
        team_list = []
        if overall_season_stats.find('li') != None:
            for specific_team_stats in overall_season_stats.find('ul').find_all('li', recursive=False):
                team = {}
                team['Team'] = specific_team_stats.a.get_text(strip=True)
                team['TeamUrl'] = specific_team_stats.a['href']
                team_list.append(team)
        specific_season['Teams'] = team_list
        season_list.append(specific_season)
    season['Seasons'] = season_list
    full_data.append(season)

In [41]:
import json

# Export data to json
def save_data (file_name, data):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

save_data("rhino_hockey_urls.json", full_data)

In [3]:
def build_data_records(season_type, season_type_url, season_stats, season_stats_url, team=None, team_url=None):
    record = []
    record.append(season_type)
    record.append(season_type_url)
    record.append(season_stats)
    record.append(season_stats_url)
    record.append(team)
    record.append(team_url)
    return record

records = []
for season_type in full_data:
    for season in season_type.get('Seasons'):
        if len(season.get('Teams')) == 0:
            records.append(build_data_records(season_type.get('SeasonType'), season_type.get('SeasonTypeUrl'), season.get('SeasonStats'), season.get('SeasonStatsUrl')))
            continue
        for team in season.get('Teams'):
            records.append(build_data_records(season_type.get('SeasonType'), season_type.get('SeasonTypeUrl'), season.get('SeasonStats'), season.get('SeasonStatsUrl'), team.get('TeamName'), team.get('TeamUrl')))

df = pd.DataFrame(records, columns=["SeasonType", "SeasonTypeUrl", "SeasonName", "SeasonUrl", "TeamName", "TeamUrl"])
df.head(100)

,SeasonType,SeasonTypeUrl,SeasonName,SeasonUrl,TeamName,TeamUrl
0,CRL WINTER PLAYOFFS,https://www.rhinohockeysiouxfalls.com/crl-wint...,20-21 CRL Winter Playoffs All Player Stats,https://www.rhinohockeysiouxfalls.com/20-21-cr...,None,None
1,CRL SUMMER PLAYOFFS,https://www.rhinohockeysiouxfalls.com/2020-crl...,2021 CRL Summer Playoffs Stats,https://www.rhinohockeysiouxfalls.com/2021-crl...,None,None
2,CRL SUMMER PLAYOFFS,https://www.rhinohockeysiouxfalls.com/2020-crl...,2020 CRL SUMMER PLAYOFFS,https://www.rhinohockeysiouxfalls.com/2020-crl...,None,None
3,CRL Winter Seasons,https://www.rhinohockeysiouxfalls.com/cenior-r...,21-22 CRL WINTER LEAGUE STATS,https://www.rhinohockeysiouxfalls.com/21-22-cr...,21-22 CAPITALS WINTER TEAM STATS,https://www.rhinohockeysiouxfalls.com/21-22-ca...
4,CRL Winter Seasons,https://www.rhinohockeysiouxfalls.com/cenior-r...,21-22 CRL WINTER LEAGUE STATS,https://www.rhinohockeysiouxfalls.com/21-22-cr...,21-22 DRAGONS WINTER TEAM STATS,https://www.rhinohockeysiouxfalls.com/21-22-dr...
...,...,...,...,...,...,...
73,CRL Spring Session,https://www.rhinohockeysiouxfalls.com/cenior-r...,2015 CRL Spring League Stats,https://www.rhinohockeysiouxfalls.com/2015-crl...,2015 Yaks Spring Team Stats,https://www.rhinohockeysiouxfalls.com/2015-yak...
74,CRL Spring Session,https://www.rhinohockeysiouxfalls.com/cenior-r...,2014 CRL Spring Team & Player Stats,https://www.rhinohockeysiouxfalls.com/2014-crl...,2014 Lightening CRL Spring Team Stats,https://www.rhinohockeysiouxfalls.com/2014-lig...
75,CRL Spring Session,https://www.rhinohockeysiouxfalls.com/cenior-r...,2014 CRL Spring Team & Player Stats,https://www.rhinohockeysiouxfalls.com/2014-crl...,2014 Rhinos CRL Spring Team Stats,https://www.rhinohockeysiouxfalls.com/2014-rhi...
76,CRL Spring Session,https://www.rhinohockeysiouxfalls.com/cenior-r...,2014 CRL Spring Team & Player Stats,https://www.rhinohockeysiouxfalls.com/2014-crl...,2014 Timbers CRL Spring Team Stats,https://www.rhinohockeysiouxfalls.com/2014-tim...


In [41]:
def get_year(season_name):
    year = re.findall(r'(^\d*-\d*|^\d{4})', season_name)[0]

    if "-" in year:
        if len(year.split("-")[0]) > 2:
            return year.split("-")[0]
        else:
            return "20" + year.split("-")[0]
    return year

def get_season(season_name):
    return None

seasons_list = ["fall", "spring", "summer", "winter"]

for season_type in full_data:
    for season in season_type.get('Seasons'):
        test = season.get('Season').split('-')
        print(int(get_year(season.get('Season'))), " | ", season.get('Season'))
        

2020  |  20-21 CRL Winter Playoffs All Player Stats
2021  |  2021 CRL Summer Playoffs Stats
2020  |  2020 CRL SUMMER PLAYOFFS
2021  |  21-22 CRL WINTER LEAGUE STATS
2020  |  20-21 CRL Winter Team & Player Stats
2019  |  2019-20 CRL Winter Player Lists
2018  |  2018-19 CRL Winter Session Stats
2017  |  2017-18 CRL Winter Season Stats
2016  |  2016-17 CRL Winter Standings
2015  |  2015-16 Cenior Rhino Winter Stats & Standings
2014  |  2014-15 CRL Winter Team & Players Stats
2021  |  2021 CRL SUMMER TEAM STATS
2020  |  2020 CRL Spring Team & Player Stats
2019  |  2019 CRL Spring Team and Player Stats
2018  |  2018 CRL  Spring Season
2017  |  2017 CRL Spring Season All Players Stats
2016  |  2016 Rhino Spring All Players Stats
2015  |  2015 CRL Spring League Stats
2014  |  2014 CRL Spring Team & Player Stats


In [58]:
sql = '''INSERT INTO dbo.RawData (SeasonTypeName,SeasonTypeUrl,SeasonName,SeasonUrl,TeamName,TeamUrl) VALUES ({}, {}, {}, {}, {}, {})'''


for data in full_data:
    if len(data) > 0:
        # Connect to the database
        cnxn = db.connect(connectionString)
        cursor = cnxn.cursor()
        # Begin attempting to insert record
        try:
            # Execute the insert SQL script
            cursor.execute(sql.format(data[0], data[1], data[2], data[3], data[4], data[5]))
            # Commit the database transaction
            cursor.commit()
        except:
            # If any errors, rollback the database transaction
            cursor.rollback()
        cursor.close()

for data in full_data:
    print(sql.format(data[0], data[1], data[2], data[3], data[4] if len(data) >= 5 else None, data[5] if len(data) >= 5 else None))